Este programa define las funciones likelihood, priors y posterior probability (quedan guardadas en Mis_funciones.py)

In [1]:
def Modelo(Mags, Phi, Me, alpha):
    """ Modelo para ajustar
    
    Parameters
    ----------
    Mags, ERR : list
        Magnitudes observadas 
    Phi, Me, alpha : .float, .float, .float
        Parámetros del modelo
        
    Returns
    --------
    F : list
        Valores de la función
    """
    import numpy as np
    M = Mags # Definición para mejor vizualización
    F = [] # Contendrá valores de la función
    ij = 0
    while ij<len(M):
        # Para que no sea tan larga la def. de "F": parto en factores a la función
        # F = f1*f2*f3
        f1 = 0.4*np.log(10)*Phi
        f2 = 10**(-0.4*(M[ij]-Me)*(alpha+1))
        f3 = np.exp( -10**(-0.4*(M[ij]-Me)) )
        
        F.append( f1*f2*f3 )
        ij = ij + 1
    return F

def Likelihood(Mags, Lum, ERR, Phi, Me, alpha):
    """ Función likelihood para el problema
    
    Parameters
    ----------
    Mags : list
        Magnitudes observadas
    Lum, ERR : list, list
        Luminosidad y sus errores asociados
    Phi, Me, alpha : .float, .float, .float
        Parámetros del modelo
        
    Returns
    --------
    LK : .float
        Valor del likelihood
    """
    import numpy as np
    import scipy.stats as st
    
    Obs = np.array(Lum)
    Calc = np.array( Modelo(Mags=Mags, Phi=Phi, Me=Me, alpha=alpha) )
    
    p = st.norm(loc=Calc, scale=ERR).pdf(Obs)
    LK = p.prod() 
    return LK

def PRIOR(Phi, Phimin, Phimax, Me, Memin, Memax, alpha, alphamin, alphamax):
    """Función prior, es un escalón en 3d
    
    Parameters
    ----------
    Phi, Phimin, Phimax : .float, .float, .float
        Valor del parámetro Phi y sus limitens inferiores y superiores para el escalón
    Me, Memin, Memax : .float, .float, .float
        Valor del parámetro Me y sus limitens inferiores y superiores para el escalón
    alpha, alphamin, alphamax : .float, .float, .float
        Valor del parámetro alpha y sus limitens inferiores y superiores para el escalón
        
    Returns
    --------
    Prob_norm: .float
        Mass probability function para el punto definido por Phi, Me y alpha
    
    """
    norm = abs(Phimax - Phimin) * abs(Memax - Memin)* abs(alphamax - alphamin)
    
    rPhi = (Phi < Phimax) * (Phi > Phimin) # Rango para Phi
    rMe = (Me < Memax) * (Me > Memin)
    ralpha = (alpha < alphamax) * (alpha > alphamin)
    print(norm, rPhi, rMe, ralpha)
    
    Prob = 1. * rPhi * rMe * ralpha
    Prob_norm = Prob/norm # Normalizo
    
    return Prob_norm

def POSTERIOR(Mags, Lum, ERR, Phi, Phimin, Phimax, Me, Memin, Memax,
              alpha, alphamin, alphamax):
    """ Devuelve el valor de la función posterior en función del likelihood y el prior
    
    Parameters
    ----------
    Tiene los mismos parámetros que las funciones Likelihood() y PRIOR()
    
    Returns
    -------
    post : .float
        Valor de la probalibidad posterior"""
    post = Likelihood(Mags, Lum, ERR, Phi, Me, alpha) * PRIOR(Phi, Phimin, Phimax, 
                                                              Me, Memin, Memax,
                                                              alpha, alphamin, alphamax)
    return post